In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import logging
import datetime

In [2]:
# 로그 생성
logger = logging.getLogger()

# 로그의 출력 기준 설정
logger.setLevel(logging.INFO)

# log 출력 형식
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# log 출력
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# log를 파일에 출력
file_handler = logging.FileHandler('D:\\통계청 공모전\\코드\\모델학습및예측(attention)_{:%Y%m%d}.log'.format(datetime.datetime.now()), 
                                   encoding='utf-8')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [3]:
df = pd.read_csv('../최종 전처리.csv')
df

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,digit_1,digit_2,digit_3,Full Text,clean_text
0,S,95.0,952.0,카센터에서 자동차부분정비 타이어오일교환,카 센터 자동차 부분 정비 타이어 오일 교환
1,G,47.0,472.0,상점내에서 일반인을 대상으로 채소 과일판매,상점 내 일반인 대상 채소 과일 판매
2,G,46.0,467.0,절단하여사업체에도매 공업용고무를가지고 합성고무도매,절단 하여사 업체 에도 매 공업 용 고무 가지 고 합성 고무 도매
3,G,47.0,475.0,영업점에서 일반소비자에게 열쇠잠금장치,영업 점 일 반 소비자 열쇠 잠금장치
4,Q,87.0,872.0,어린이집 보호자의 위탁을 받아 취학전아동보육,어린이집 보호자 위탁 받아 취학 전 아동 보육
...,...,...,...,...,...
1099995,NaN,NaN,NaN,사업장에서 일반인대상으로 버섯농장,사업 장 일반인 대상 버섯 농장
1099996,NaN,NaN,NaN,한의원에서 외래환자위주고 치료,한의원 외래 환자 위주 고 치료
1099997,NaN,NaN,NaN,일반점포에서 소비자에게 그림판매,일 반 점포 소비자 그림 판매
1099998,NaN,NaN,NaN,사업장에서 일반인 학생대상으로 학습공간제공,사업 장 일반인 학생 대상 학습 공간 제공


In [4]:
train = df[:1000000]
train

,digit_1,digit_2,digit_3,Full Text,clean_text
0,S,95.0,952.0,카센터에서 자동차부분정비 타이어오일교환,카 센터 자동차 부분 정비 타이어 오일 교환
1,G,47.0,472.0,상점내에서 일반인을 대상으로 채소 과일판매,상점 내 일반인 대상 채소 과일 판매
2,G,46.0,467.0,절단하여사업체에도매 공업용고무를가지고 합성고무도매,절단 하여사 업체 에도 매 공업 용 고무 가지 고 합성 고무 도매
3,G,47.0,475.0,영업점에서 일반소비자에게 열쇠잠금장치,영업 점 일 반 소비자 열쇠 잠금장치
4,Q,87.0,872.0,어린이집 보호자의 위탁을 받아 취학전아동보육,어린이집 보호자 위탁 받아 취학 전 아동 보육
...,...,...,...,...,...
999995,C,13.0,134.0,제품입고 워싱 청바지워싱,제품 입고 워싱 청바지 워싱
999996,F,42.0,424.0,현장에서 고객의요청에의해 실내인테리어,현장 고객 요청 에의 해 실내 인테리어
999997,G,47.0,474.0,영업점에서 일반소비자에게 여성의류 판매,영업 점 일 반 소비자 여성 의류 판매
999998,P,85.0,856.0,사업장에서 일반고객을대상으로 필라테스,사업 장 일 반 고객 대상 필라테스


In [5]:
encoder1 = LabelEncoder()
train['digit_1'] = encoder1.fit_transform(train['digit_1'])
train['digit_1'].nunique()

C:\Users\user\AppData\Local\Temp/ipykernel_19480/3867300257.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['digit_1'] = encoder1.fit_transform(train['digit_1'])


19

In [6]:
encoder2 = LabelEncoder()
train['digit_2'] = encoder2.fit_transform(train['digit_2'])
train['digit_2'].nunique()

C:\Users\user\AppData\Local\Temp/ipykernel_19480/2282110248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['digit_2'] = encoder2.fit_transform(train['digit_2'])


74

In [7]:
encoder3 = LabelEncoder()
train['digit_3'] = encoder3.fit_transform(train['digit_3'])
train['digit_3'].nunique()

C:\Users\user\AppData\Local\Temp/ipykernel_19480/781095071.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['digit_3'] = encoder3.fit_transform(train['digit_3'])


225

In [8]:
text = np.load('D:/통계청 공모전/text.npy', allow_pickle=True)

# 시퀀스 패딩

In [9]:
logging.info('시퀀스 패딩')

2022-04-13 02:38:07,312 - root - INFO - 시퀀스 패딩


In [10]:
max_len = 21 # 단어 개수 최댓값

text = tf.keras.preprocessing.sequence.pad_sequences(text, maxlen=max_len)
text

array([[   0,    0,    0, ...,  633, 1945, 3832],
       [   0,    0,    0, ...,  296,  147,   12],
       [   0,    0,    0, ..., 1304,  608,   23],
       ...,
       [   0,    0,    0, ..., 1444, 1030,   12],
       [   0,    0,    0, ..., 1434, 1726,   49],
       [   0,    0,    0, ...,   44, 1310,  203]])

In [11]:
train_data = text[:1000000]
test_data = text[1000000:]
target1 = train['digit_1']

X_train, X_test, y_train, y_test = train_test_split(train_data, target1, test_size=0.2, shuffle=True, stratify=target1)

In [12]:
X_train

array([[    0,     0,     0, ..., 10054,  2285,   462],
       [    0,     0,     0, ...,    67,   158,    79],
       [    0,     0,     0, ...,   779,  2897,  4509],
       ...,
       [    0,     0,     0, ...,    95, 10049,   255],
       [    0,     0,     0, ...,     0,  5890,   258],
       [    0,     0,     0, ...,     0, 10166,  1009]])

# 멀티 헤드 어텐션

In [13]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'embedding_dim': 'self.embedding_dim',
            'num_heads': 'self.num_heads',
            'projection_dim': 'self.projection_dim',
            'query_dense': 'self.query_dense',
            'key_dense': 'self.key_dense',
            'value_dense': 'self.value_dense',
            'dense': 'self.dense'
        })
        return config

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs

# 인코더 설계

In [14]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'att': 'self.att',
            'ffn': 'self.ffn',
            'layernorm1': 'self.layernorm1',
            'layernorm2': 'self.layernorm2',
            'dropout1': 'self.dropout1',
            'dropout2': 'self.dropout2'
        })
        return config

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm

# 포지션 임베딩

In [15]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_emb = tf.keras.layers.Embedding(max_len, embedding_dim)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'token_emb': 'self.token_emb',
            'pos_emb': 'self.pos_emb'
        })
        return config
        
    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

# Attention 학습 (digit_1)

In [16]:
logging.info('digit_1 학습')

2022-04-13 02:38:10,442 - root - INFO - digit_1 학습


In [17]:
embedding_dim = 128  # 각 단어의 임베딩 벡터의 차원
num_heads = 2  # 어텐션 헤드의 수
dff = 128  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
vocab_size = 10760# sentencepiece와 동일
max_len = 21 # 텍스트 최대 길이는 21

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(19, activation="softmax")(x) # digit_1 19개 카테고리

model1 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [18]:
es = EarlyStopping(monitor='accuracy', mode = 'auto', verbose=3, patience=4)
mc1 = ModelCheckpoint('best_model1.h5', monitor='val_accuracy', mode = 'auto', verbose=1, save_best_only=True)

In [19]:
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history1 = model1.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc1], validation_split=0.2)

Epoch 1/10
20000/20000 [==============================] - 277s 14ms/step - loss: 0.1736 - accuracy: 0.9555 - val_loss: 0.1156 - val_accuracy: 0.9691

Epoch 00001: val_accuracy improved from -inf to 0.96909, saving model to best_model1.h5
Epoch 2/10
20000/20000 [==============================] - 276s 14ms/step - loss: 0.1150 - accuracy: 0.9698 - val_loss: 0.1095 - val_accuracy: 0.9711

Epoch 00002: val_accuracy improved from 0.96909 to 0.97109, saving model to best_model1.h5
Epoch 3/10
20000/20000 [==============================] - 275s 14ms/step - loss: 0.1029 - accuracy: 0.9726 - val_loss: 0.1088 - val_accuracy: 0.9713

Epoch 00003: val_accuracy improved from 0.97109 to 0.97130, saving model to best_model1.h5
Epoch 4/10
20000/20000 [==============================] - 278s 14ms/step - loss: 0.0963 - accuracy: 0.9742 - val_loss: 0.1048 - val_accuracy: 0.9723

Epoch 00004: val_accuracy improved from 0.97130 to 0.97229, saving model to best_model1.h5
Epoch 5/10
20000/20000 [===============

In [20]:
print("테스트 정확도: %.4f" % (model1.evaluate(X_test, y_test)[1]))

6250/6250 [==============================] - 22s 3ms/step - loss: 0.1043 - accuracy: 0.9725
테스트 정확도: 0.9725


In [21]:
pred1 = model1.predict(X_test)
pred1

array([[3.6744474e-09, 3.6862422e-23, 2.5232512e-09, ..., 4.9089958e-06,
        9.6294284e-01, 6.7912370e-05],
       [2.0091236e-11, 2.5880529e-28, 3.2671644e-06, ..., 4.0614084e-10,
        7.1699736e-11, 1.9643352e-07],
       [2.3981828e-13, 2.6054717e-25, 4.3806290e-07, ..., 1.1937420e-09,
        5.3928972e-09, 4.3856162e-05],
       ...,
       [5.4284908e-14, 2.3214447e-28, 3.5151174e-08, ..., 9.9999118e-01,
        5.8357130e-12, 8.7233120e-06],
       [1.5016700e-01, 1.0709922e-10, 5.9273603e-05, ..., 1.7090443e-03,
        8.6474710e-04, 2.2870956e-03],
       [9.5380046e-06, 3.8843538e-15, 4.9531832e-03, ..., 1.4446202e-06,
        1.2500817e-07, 2.2345599e-05]], dtype=float32)

In [22]:
predict1 = []
for i in range(pred1.shape[0]):
    predict1.append(pred1[i].argmax())
predict1 = np.array(predict1)
predict1

array([17,  6, 11, ..., 16, 14,  6], dtype=int64)

In [23]:
print ('Here is the classification report:')
print (classification_report(y_test, predict1))

Here is the classification report:
              precision    recall  f1-score   support

           0       0.77      0.75      0.76       213
           1       0.90      0.84      0.87        85
           2       0.95      0.96      0.96     21038
           3       0.83      0.91      0.86       151
           4       0.87      0.83      0.85       451
           5       0.94      0.95      0.94      7010
           6       0.98      0.98      0.98     49294
           7       0.99      0.99      0.99     19608
           8       0.99      0.99      0.99     37485
           9       0.92      0.93      0.92      2172
          10       0.98      0.97      0.97      2076
          11       0.98      0.98      0.98      8028
          12       0.94      0.92      0.93      5687
          13       0.89      0.86      0.87      3540
          14       0.91      0.92      0.91       593
          15       0.97      0.97      0.97      9322
          16       0.96      0.98      0.97   

# 모델개발용 데이터 예측 (digit_1)

In [24]:
logging.info('모델개발용 자료 digit_1 예측')

2022-04-13 03:25:02,723 - root - INFO - 모델개발용 자료 digit_1 예측


In [25]:
digit1_pred = model1.predict(test_data)
digit1_pred

array([[2.7055127e-17, 1.2330838e-30, 1.1963772e-06, ..., 1.7763506e-09,
        2.4055899e-09, 2.0344249e-10],
       [3.8351571e-09, 5.0636559e-23, 1.4673577e-04, ..., 1.0049971e-09,
        1.4671987e-09, 5.0718063e-07],
       [1.5753430e-12, 2.8529792e-24, 6.4361379e-06, ..., 1.2847735e-04,
        4.8474963e-06, 9.9981493e-01],
       ...,
       [1.4143646e-07, 6.1266792e-19, 2.1509189e-04, ..., 3.7780251e-07,
        9.2971635e-07, 1.6625205e-05],
       [3.9354620e-07, 2.7546263e-16, 1.3227590e-06, ..., 1.3239908e-03,
        7.8114200e-01, 1.0548651e-02],
       [6.5728287e-08, 1.5419269e-16, 1.7324975e-05, ..., 5.2463741e-07,
        2.7765286e-06, 2.8492982e-04]], dtype=float32)

In [26]:
digit1_predict = []
for i in range(digit1_pred.shape[0]):
    digit1_predict.append(digit1_pred[i].argmax())
digit1_predict = np.array(digit1_predict)
digit1_predict

array([ 8,  6, 18, ...,  6, 17, 11], dtype=int64)

In [27]:
digit1_predict = encoder1.inverse_transform(digit1_predict)
digit1_predict

array(['I', 'G', 'S', ..., 'G', 'R', 'L'], dtype=object)

# Attention 학습 (digit_2)

In [28]:
logging.info('digit_2 학습')

2022-04-13 03:25:09,682 - root - INFO - digit_2 학습


In [29]:
target2 = train['digit_2']
# digit_2에 1개만 가지고 있는 카테고리 존재하므로 stratify=target2하면 에러 발생
X_train, X_test, y_train, y_test = train_test_split(train_data, target2, test_size=0.2, shuffle=True)

In [30]:
embedding_dim = 128  # 각 단어의 임베딩 벡터의 차원
num_heads = 2  # 어텐션 헤드의 수
dff = 128  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
vocab_size = 10760# sentencepiece와 동일
max_len = 21 # 텍스트 최대 길이는 21

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(74, activation="softmax")(x) # digit_2 74개 카테고리

model2 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [31]:
# earlystopping은 위에 정의되었으므로 checkpoint만 정의
mc2 = ModelCheckpoint('best_model2.h5', monitor='val_accuracy', mode = 'auto', verbose=1, save_best_only=True)

In [32]:
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history2 = model2.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc2], validation_split=0.2)

Epoch 1/10
20000/20000 [==============================] - 286s 14ms/step - loss: 0.4044 - accuracy: 0.8988 - val_loss: 0.2709 - val_accuracy: 0.9269

Epoch 00001: val_accuracy improved from -inf to 0.92686, saving model to best_model2.h5
Epoch 2/10
20000/20000 [==============================] - 286s 14ms/step - loss: 0.2825 - accuracy: 0.9262 - val_loss: 0.2523 - val_accuracy: 0.9316

Epoch 00002: val_accuracy improved from 0.92686 to 0.93158, saving model to best_model2.h5
Epoch 3/10
20000/20000 [==============================] - 285s 14ms/step - loss: 0.2545 - accuracy: 0.9323 - val_loss: 0.2427 - val_accuracy: 0.9338

Epoch 00003: val_accuracy improved from 0.93158 to 0.93378, saving model to best_model2.h5
Epoch 4/10
20000/20000 [==============================] - 285s 14ms/step - loss: 0.2384 - accuracy: 0.9363 - val_loss: 0.2430 - val_accuracy: 0.9364

Epoch 00004: val_accuracy improved from 0.93378 to 0.93641, saving model to best_model2.h5
Epoch 5/10
20000/20000 [===============

In [33]:
print("테스트 정확도: %.4f" % (model2.evaluate(X_test, y_test)[1]))

6250/6250 [==============================] - 21s 3ms/step - loss: 0.2487 - accuracy: 0.9369
테스트 정확도: 0.9369


In [34]:
pred2 = model2.predict(X_test)
pred2

array([[8.0574332e-24, 1.0459219e-35, 2.6236847e-36, ..., 2.3963528e-18,
        3.7517708e-11, 6.0947419e-12],
       [1.1395154e-09, 3.1561897e-17, 2.4935400e-20, ..., 5.8731486e-09,
        1.3013868e-06, 1.1577843e-06],
       [2.5615198e-06, 3.5633482e-12, 9.2864583e-13, ..., 6.8958349e-07,
        3.4945526e-05, 4.3379541e-06],
       ...,
       [8.0337458e-08, 3.2540022e-20, 1.0309625e-12, ..., 9.9039418e-08,
        1.7358650e-07, 1.2801241e-10],
       [7.4235774e-12, 6.7201246e-24, 5.3819610e-17, ..., 7.9964689e-12,
        1.7829249e-10, 3.9351780e-09],
       [1.1130115e-05, 1.7127410e-16, 1.3721958e-11, ..., 1.7990868e-06,
        2.0121060e-09, 2.5641193e-06]], dtype=float32)

In [35]:
predict2 = []
for i in range(pred2.shape[0]):
    predict2.append(pred2[i].argmax())
predict2 = np.array(predict2)
predict2

array([42, 41, 41, ..., 40, 47,  7], dtype=int64)

In [36]:
print ('Here is the classification report:')
print (classification_report(y_test, predict2))

Here is the classification report:
              precision    recall  f1-score   support

           0       0.75      0.69      0.72       146
           1       0.78      0.82      0.80        22
           2       0.80      0.67      0.73        18
           4       0.00      0.00      0.00         2
           5       0.77      0.82      0.80        88
           7       0.91      0.91      0.91      2896
           8       0.77      0.62      0.68        81
          10       0.85      0.81      0.83      1060
          11       0.87      0.92      0.89      1082
          12       0.88      0.78      0.83       265
          13       0.66      0.79      0.72       316
          14       0.82      0.78      0.80       370
          15       0.89      0.88      0.89       815
          16       0.67      0.40      0.50        20
          17       0.72      0.60      0.66       657
          18       0.57      0.16      0.25        83
          19       0.68      0.76      0.71   

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 모델개발용 데이터 예측 (digit_2)

In [37]:
logging.info('모델개발용 자료 digit_2 예측')

2022-04-13 04:13:18,696 - root - INFO - 모델개발용 자료 digit_2 예측


In [38]:
digit2_pred = model2.predict(test_data)
digit2_pred

array([[6.44739817e-12, 9.83577892e-27, 6.82150167e-18, ...,
        2.50037231e-12, 9.66002712e-12, 1.10629936e-10],
       [4.82672213e-08, 1.31892915e-18, 8.15208914e-16, ...,
        1.35204319e-08, 6.00098602e-08, 6.82187806e-10],
       [1.91612894e-08, 4.37279053e-19, 1.86410827e-14, ...,
        9.99920487e-01, 1.67164527e-14, 6.89315243e-07],
       ...,
       [4.49446702e-10, 6.98911700e-18, 8.07844963e-22, ...,
        1.62120473e-09, 3.24203342e-07, 7.65308926e-07],
       [6.88958959e-11, 7.63807293e-08, 1.30734490e-17, ...,
        3.07126157e-03, 3.06107015e-11, 6.35455654e-04],
       [4.66152182e-13, 1.14733694e-15, 6.95590031e-23, ...,
        1.96713845e-05, 3.24795940e-10, 7.49731726e-08]], dtype=float32)

In [39]:
digit2_predict = []
for i in range(digit2_pred.shape[0]):
    digit2_predict.append(digit2_pred[i].argmax())
digit2_predict = np.array(digit2_predict)
digit2_predict

array([47, 40, 71, ..., 41, 69, 57], dtype=int64)

In [40]:
digit2_predict = encoder2.inverse_transform(digit2_predict)
digit2_predict

array([56., 46., 94., ..., 47., 90., 68.])

# Attention 학습 (digit_3)

In [41]:
logging.info('digit_3 학습')

2022-04-13 04:13:25,449 - root - INFO - digit_3 학습


In [42]:
target3 = train['digit_3']
# digit_3에 1개만 가지고 있는 카테고리 존재하므로 stratify=target3하면 에러 발생
X_train, X_test, y_train, y_test = train_test_split(train_data, target3, test_size=0.2, shuffle=True)

In [43]:
embedding_dim = 128  # 각 단어의 임베딩 벡터의 차원
num_heads = 2  # 어텐션 헤드의 수
dff = 128  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
vocab_size = 10760# sentencepiece와 동일
max_len = 21 # 텍스트 최대 길이는 21

inputs = tf.keras.layers.Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
x = transformer_block(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(20, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(225, activation="softmax")(x) # digit_3 225개 카테고리

model3 = tf.keras.Model(inputs=inputs, outputs=outputs)

In [44]:
# earlystopping은 위에 정의되었으므로 checkpoint만 정의
mc3 = ModelCheckpoint('best_model3.h5', monitor='val_accuracy', mode = 'auto', verbose=1, save_best_only=True)

In [45]:
model3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history3 = model3.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc3], validation_split=0.2)

Epoch 1/10
20000/20000 [==============================] - 281s 14ms/step - loss: 0.6882 - accuracy: 0.8374 - val_loss: 0.4336 - val_accuracy: 0.8902

Epoch 00001: val_accuracy improved from -inf to 0.89018, saving model to best_model3.h5
Epoch 2/10
20000/20000 [==============================] - 281s 14ms/step - loss: 0.4764 - accuracy: 0.8821 - val_loss: 0.4078 - val_accuracy: 0.8987

Epoch 00002: val_accuracy improved from 0.89018 to 0.89866, saving model to best_model3.h5
Epoch 3/10
20000/20000 [==============================] - 280s 14ms/step - loss: 0.4322 - accuracy: 0.8926 - val_loss: 0.3877 - val_accuracy: 0.9031

Epoch 00003: val_accuracy improved from 0.89866 to 0.90307, saving model to best_model3.h5
Epoch 4/10
20000/20000 [==============================] - 280s 14ms/step - loss: 0.4074 - accuracy: 0.8983 - val_loss: 0.3805 - val_accuracy: 0.9052

Epoch 00004: val_accuracy improved from 0.90307 to 0.90520, saving model to best_model3.h5
Epoch 5/10
20000/20000 [===============

In [46]:
print("테스트 정확도: %.4f" % (model3.evaluate(X_test, y_test)[1]))

6250/6250 [==============================] - 21s 3ms/step - loss: 0.3796 - accuracy: 0.9082
테스트 정확도: 0.9082


In [47]:
pred3 = model3.predict(X_test)
pred3

array([[8.5577856e-18, 2.1199641e-24, 2.6724954e-16, ..., 1.1623575e-10,
        6.0267655e-07, 2.0534596e-08],
       [5.4744869e-21, 6.7136738e-29, 2.4174795e-17, ..., 1.9860389e-18,
        9.2810850e-13, 7.3175160e-10],
       [4.4705647e-26, 2.0512989e-34, 1.9087159e-23, ..., 1.5414659e-18,
        9.9999988e-01, 4.8726982e-09],
       ...,
       [7.5610559e-12, 3.2844449e-13, 1.8069662e-11, ..., 3.9933337e-04,
        5.7626945e-05, 7.8125631e-05],
       [1.2891334e-09, 4.1403347e-18, 1.7705743e-08, ..., 7.3475235e-08,
        9.6909480e-06, 8.1492288e-07],
       [6.0042566e-20, 2.2408139e-19, 8.6936173e-21, ..., 8.3587710e-13,
        2.2902345e-10, 4.7995630e-09]], dtype=float32)

In [48]:
predict3 = []
for i in range(pred3.shape[0]):
    predict3.append(pred3[i].argmax())
predict3 = np.array(predict3)
predict3

array([114, 204, 223, ..., 216, 200, 147], dtype=int64)

In [49]:
print ('Here is the classification report:')
print (classification_report(y_test, predict3))

Here is the classification report:


C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.65      0.63      0.64        81
           1       0.95      0.73      0.82        51
           2       0.73      0.30      0.42        37
           3       0.73      0.65      0.69        17
           4       0.00      0.00      0.00         3
           5       0.89      0.64      0.74        25
           6       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         2
           9       0.69      0.71      0.70        35
          10       0.96      0.90      0.93        52
          11       0.00      0.00      0.00         2
          12       0.82      0.69      0.75       147
          13       0.86      0.84      0.85       215
          14       0.67      0.63      0.65       231
          15       0.91      0.85      0.88       172
          16       0.67      0.11      0.18        19
          17       0.80      0.75      0.77       279
          18       0.84    

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 모델개발용 데이터 예측 (digit_3)

In [50]:
logging.info('모델개발용 자료 digit_3 예측')

2022-04-13 05:00:45,593 - root - INFO - 모델개발용 자료 digit_3 예측


In [51]:
digit3_pred = model3.predict(test_data)
digit3_pred

array([[6.2563634e-17, 1.9609120e-15, 3.0460039e-17, ..., 4.3088935e-11,
        3.1479285e-08, 1.2519715e-07],
       [1.5971837e-16, 2.0712687e-21, 5.5032541e-18, ..., 7.8964729e-10,
        1.1279634e-09, 1.6028427e-11],
       [6.3551639e-22, 2.7224877e-23, 4.8309474e-18, ..., 2.7037105e-14,
        5.0249679e-12, 1.7237244e-06],
       ...,
       [2.4316460e-10, 3.9914166e-12, 1.2786231e-11, ..., 3.9645296e-05,
        1.6685601e-05, 3.3624217e-04],
       [6.9839100e-13, 4.3458348e-11, 2.4206608e-09, ..., 1.7648704e-08,
        5.3350999e-07, 7.0620104e-07],
       [2.3349277e-11, 1.3090670e-14, 2.5812559e-09, ..., 5.2324722e-09,
        9.3080160e-10, 1.6271955e-05]], dtype=float32)

In [52]:
digit3_predict = []
for i in range(digit3_pred.shape[0]):
    digit3_predict.append(digit3_pred[i].argmax())
digit3_predict = np.array(digit3_predict)
digit3_predict

array([147, 122, 219, ..., 132, 214, 169], dtype=int64)

In [53]:
digit3_predict = encoder3.inverse_transform(digit3_predict)
digit3_predict

array([561., 466., 949., ..., 478., 902., 682.])

# 답안 작성용 파일

In [54]:
submission = pd.read_csv('../답안 작성용 파일.csv', encoding='cp949')
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [55]:
submission['digit_1'] = digit1_predict
submission['digit_2'] = digit2_predict
submission['digit_3'] = digit3_predict

In [56]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56.0,561.0,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46.0,466.0,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94.0,949.0,절에서,신도을 대상으로,불교단체운영
3,id_000004,G,95.0,952.0,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56.0,562.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,47.0,472.0,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86.0,862.0,한의원에서,외래환자위주고,치료
99997,id_099998,G,47.0,478.0,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90.0,902.0,사업장에서,일반인.학생대상으로,학습공간제공


In [57]:
submission.to_csv('답안 작성용 파일(attention).csv', index=False, encoding='utf-8-sig')